# Scoring Neighbourhoods

The scoring methodology is based on 4 variables (household income, house prices, % of "favourable" occupations, % of graduates) which are combined using PCA to score each neighbourhood.

The below code loads in relevant function libraries and the data to build the SES scores.

In [ ]:
# Needed on a Mac
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt 

In [ ]:
# For reproducibility
import random
import numpy as np
r_state = 42
random.seed(r_state) 
np.random.seed(r_state)

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import pysal as ps
import seaborn as sns

from sklearn import decomposition  
from sklearn.preprocessing import scale  
from sklearn import preprocessing 
from sklearn import linear_model
from sklearn import model_selection
#from sklearn import cross_validation

from scipy.stats import boxcox
from scipy.stats import spearmanr
from scipy.stats import pearsonr

In [ ]:
lkp = os.path.join('data','lkp')
src = os.path.join('data','src')

analytical = os.path.join('data','analytical')
canonical  = os.path.join('data','canonical')
converted  = os.path.join(canonical,'converted')
greenspace = os.path.join(canonical,'greenspace')
dwelling   = os.path.join(canonical,'dwellings')
travel     = os.path.join(canonical,'travel')
household  = os.path.join(canonical,'households')
housing    = os.path.join(canonical,'housing')
work       = os.path.join(canonical,'work')
scores     = os.path.join(canonical,'scores')

for d in [analytical,canonical,converted,greenspace,dwelling,travel,household,housing,work,scores]:
    if not os.path.exists(d):
        os.makedirs(d)

In [ ]:
def plot_checks(df, selected_cols=None, prefix='Test'):
    sns.set(rc={"figure.figsize": (12, 3)})
    if not selected_cols:
        selected_cols = df.columns
    for d in selected_cols:
        print("Working on " + d)
        fig = plt.figure(d)
        sns.distplot(df[d], color='green', hist=True, rug=True, norm_hist=False)
        fig = plt.gcf() # *G*et the *C*urrent *F*igure environment so that the next command works
        plt.savefig("{0}-{1}-Check.pdf".format(prefix, d.replace(':',' - ')), bbox_inches="tight")
        plt.close()
    print("Done.")
    return

## Load Scoring Data for 2001 and 2011

### Build the Scoring Data Set

In [ ]:
df11 = pd.DataFrame()
df01 = pd.DataFrame()
for d in ['Income.csv','Values.csv','Occupations.csv','Qualifications.csv']:
    if 'Values' in d:
        loc = housing
    else:
        loc = work
    tmp11_df = pd.read_csv(os.path.join(loc,d.replace('.csv','-2011.csv')))
    tmp01_df = pd.read_csv(os.path.join(loc,d.replace('.csv','-2001.csv')))
    
    # Needed for GeoConvert-ed data, will have no effect on other dfs
    tmp01_df.rename(columns=
                  {'2011 census lower super output areas and data zones  (2001 codes used in scotland)':'lsoacd'},
                 inplace=True)
    
    if df11.shape[0] == 0:
        df11 = tmp11_df
        df01 = tmp01_df
    else: 
        df11 = pd.merge(df11, tmp11_df, how='outer', left_on='lsoacd', right_on='lsoacd')
        df01 = pd.merge(df01, tmp01_df, how='outer', left_on='lsoacd', right_on='lsoacd')

print("Shape of 2001 data frame: " + str(df01.shape))
print("Shape of 2011 data frame: " + str(df11.shape))

In [ ]:
rename = {
    'Mean':'Mean_inc',
    'Median_x':'Median_inc',
    'Median_y':'Median_hp',
    'Total_x':'Total (Occupations)',
    'Total_y':'Total (Qualifications)'
}
df01.rename(columns=rename, inplace=True)
df11.rename(columns=rename, inplace=True)
print("Columns renamed to remove ambiguity.")

#  Set the index of dataframe to LSOA
df01.set_index('lsoacd', inplace=True)
df11.set_index('lsoacd', inplace=True)
print("Datasets indexed to LSOA")

df11.sample(3, random_state=r_state)

## Transform Property Prices and Income

These are so heavily skewed that some kind of transformation may be required to prevent high prices or incomes dominating the scoring metric. This is something of a judgement call, so for completeness we've included three possible ways of working with the data but have selected only one of them for our analysis.

<span style="color:red;font-weight:bolder;size:14pt;">You can run _all_ of the following options, but you can only select _one_ for further work in the PCA Processing section.</span>

### Option 1: No Transform

In this option we pass through household income and property price 'as is'. These will consequently weigh more heavily in the final score and highlight slightly different features across London.

In [ ]:
df_transformed = pd.DataFrame({
    'hpu_01' : df01['Median Property Price'],
    'hpu_11' : df11['Median Property Price'],
    'hhu_01' : df01['Median Income'],
    'hhu_11' : df11['Median Income']
}, index=df01.index)

### Option 2: Box-Cox Transform

In this option with use Box-Cox transforms on these two variables so that they are pseudo-normal. Note that we need to use the same transform on both years so as to ensure that the results are comparable across 2001/2011. Since skills and occupation remain skewed (albeit much less so) this will tend to highlight changes in 'human factor' issues and deemphasise financial changes.

In [ ]:
# We want to use the same transform so that change is detectable
hpb01, lmd01a = boxcox(df01['Median Property Price'])  #  Process 2001 data
print("2001 property price transform lambda: " + str(lmd01a))
hpb11 = boxcox(df11['Median Property Price'], lmbda=lmd01a)
print("Property prices transformed using same Box-Cox lambda.")

print(" ")

hhb01, lmd01b = boxcox(df01['Median Income'])  #  Process 2001 data
print("2001 income transform lambda: " + str(lmd01b))
hhb11 = boxcox(df11['Median Income'], lmbda=lmd01b)
print("Household income transformed using same Box-Cox lambda.")

df_transformed2 = pd.DataFrame({
    'hpb_01': hpb01,
    'hpb_11': hpb11,
    'hhb_01': hhb01,
    'hhb_11': hhb11
}, index=df01.index)

In [ ]:
df_transformed = pd.merge(df_transformed, df_transformed2, how='inner', left_index=True, right_index=True)

### Option 3: In-Between Transform

Transforming for normality has quite a strong impact on the score, so possibly it would be better to select an intermeidate transformation that has less of an impact. Such as a fairly common log-transform on prices, and $x^{2/3}$ on incomes.

In [ ]:
# We want to use the same transform so that change is detectable
hpl01 = np.log(df01['Median Property Price'])  #  Process 2001 data
hpl11 = np.log(df11['Median Property Price'])
print("Property prices transformed using natural log.")

print(" ")

hhl01 = np.power(df01['Median Income'], 2.0/3.0)  #  Process 2001 
hhl11 = np.power(df11['Median Income'], 2.0/3.0)
print("Household income transformed using same exponent.")

df_transformed3 = pd.DataFrame({
    'hpl_01': hpl01,
    'hpl_11': hpl11,
    'hhl_01': hhl01,
    'hhl_11': hhl11
}, index=df01.index)

In [ ]:
df_transformed = pd.merge(df_transformed, df_transformed3, how='inner', left_index=True, right_index=True)
print("Final shape: " + str(df_transformed.shape))

In [ ]:
# Slow. Uncomment to explore further.
#plot_checks(df_transformed, prefix='Transforms')
print("Done printing out results of transforms.")

In [ ]:
df_transformed.describe()

## Calculate Occupation Share

Each LSOA is assessed on the share of occupations in terms of their likely contribution to neighbourhood change. Note that shares are not transformed as they are much less likely to be as heavily skewed.

In [ ]:
#  Process occupational data
def process_occ_data(df):
    #  Columns of interest
    occ = ['Managerial','Professional','Technical','Administrative','Skilled','Personal Service','Customer Service','Operators','Elementary']
    
    # Integrate results into Occupations datasets -- 
    # right now we don't replicate Jordan's approach of
    # grouping them into 'knowledge worker' and 'other'. 
    # We've added calculation of the HHI since it's a nice 
    # measure of diversity and might help us to unpick
    # different types of change.
    occ_data = pd.DataFrame()
    for c in occ:
        # Only if we want percentages for each group
        #occ_data[c+'_pct']   = (df.loc[:,c] / df.loc[:,'Total_occ'])*100
        occ_data[c+'_share'] = (df.loc[:,c] / df.loc[:,'Total (Occupations)'])**2
    
    # Calculate the HHI to get at dominance by a group/trade
    #occ_data['hhi_occ'] = occ_data[[s for s in occ_data.columns if '_share' in s]].sum(axis=1)
    
    # Drop the share columns
    occ_data.drop([s for s in occ_data.columns if '_share' in s], axis=1, inplace=True)
    
    # Add the 'knowledge worker' share -- this is columns 0-2 of the data frame
    occ_data['kw_pct'] = (df.loc[:,occ[0:3]].sum(axis=1) / df.loc[:,'Total (Occupations)'])*100
    
    return occ_data

occ01 = process_occ_data(df01)  #  Processed 2001 occupation data
occ11 = process_occ_data(df11)  #  Processed 2011 occupation data

print("Allows you to examine how occupations data will be processed.")

In [ ]:
occ01.sample(3, random_state=r_state)

In [ ]:
#g = sns.jointplot(y=occ01.hhi_occ, x=occ01.kw_pct, kind='kde', stat_func=spearmanr)

## Calculate Qualification Share

Each LSOA is scored on the % of graduates (or equivalent), which corresponds to "level 4-5" in the Census methodology. Note that shares are not transformed at this stage as they're much less likely to be heavily skewed.

In [ ]:
#  Process qualifications data
def process_quals_data(df):
    # Columns of interest
    quals  = ['No qualifications','Level 1','Level 2','Level 3','Level 4 and above','Other']
    squals = ['Students In employment','Students Unemployed','Students inactive'] # Not currently used
    
    #  Integrate results into Qualifications datasets  -- 
    # right now we don't replicate Jordan's approach of
    # grouping them into 'highly educated' and 'other'. 
    # We've added calculation of the HHI since it's a nice 
    # measure of diversity and might help us to unpick
    # different types of change.
    quals_data = pd.DataFrame()
    for c in quals:
        #quals_data[c+'_pct']   = (df.loc[:,c] / df.loc[:,'Total_qual'])*100
        quals_data[c+'_share'] = (df.loc[:,c] / df.loc[:,'Total (Qualifications)'])**2
    
    # Calculate the HHI to get at dominance by a group/trade
    #quals_data['hhi_quals'] = quals_data[[s for s in quals_data.columns if '_share' in s]].sum(axis=1)
    
    # Drop the share columns
    quals_data.drop([s for s in quals_data.columns if '_share' in s], axis=1, inplace=True)
    
    # The 'highly educated' share -- this is columns 0-2 of the data frame
    quals_data['he_pct'] = (df.loc[:,quals[4]] / df.loc[:,'Total (Qualifications)'])*100
    
    return quals_data

qual01 = process_quals_data(df01)  #  Qualification data 2001
qual11 = process_quals_data(df11)  #  Qualification data 2011 

print("Allows you to examine how qualifications data will be processed.")

In [ ]:
qual01.sample(3, random_state=r_state)

In [ ]:
#g = sns.jointplot(y=qual01.hhi_quals, x=qual01.he_pct, kind='kde', stat_func=spearmanr)

## Checking on Distributions of Other Variables

In [ ]:
quals_01 = process_quals_data(df01)
quals_11 = process_quals_data(df11)
occ_01   = process_occ_data(df01)
occ_11   = process_occ_data(df11)

df_test = pd.concat([quals_01, quals_11, occ_01, occ_11], join='inner', axis=1)
df_test.columns = ['Qualifications 2001','Qualifications 2011','Occupations 2001','Occupations 2011']
df_test.sample(3, random_state=r_state)

In [ ]:
# Slow. Uncomment to explore futher.
#plot_checks(df_test, prefix='Transforms')

## PCA Processing

PCA was used to combine data on the four variables into a single score.  First, Principal Components Analysis finds the single component which maximises the variance in the data.  Data for the scoring variables from both the 2001 and 2011 datasets were combined (vertically) into a single dataset which was used to find the PCA transformation.

<span style="color:red;font-weight:bolder">You will have a choice to make here: which transform (if any) do you want to use on the skewed data? Note that your choice here should be the same as the one used in notebook 6; there is nothing, however, preventing your running this next section once using each type of transform so that you output all three scoring files and can more quickly experiment with different options in notebook 6.</span>

In [ ]:
# Don't forget to set the house price and income scores to the 
# transform/non-transform that you want to use!
to_use = 'Untransformed' # Choices: ['Untransformed','Box-Cox','Log']

In [ ]:
#  Indicators
quals_score_01    = process_quals_data(df01)  #  Qualifications
occ_score_01      = process_occ_data(df01)    #  Occupation
quals_score_11    = process_quals_data(df11)  #  Qualifications
occ_score_11      = process_occ_data(df11)    #  Occupation

house_pr_score_01 = None  #  House Prices
hh_inc_score_01   = None  #  Household Income
house_pr_score_11 = None  #  House Prices
hh_inc_score_11   = None  #  Household income

if to_use == 'Untransformed':
    house_pr_score_01 = df_transformed['hpu_01']  #  House Prices
    hh_inc_score_01   = df_transformed['hhu_01']  #  Household Income
    house_pr_score_11 = df_transformed['hpu_11']  #  House Prices
    hh_inc_score_11   = df_transformed['hhu_11']  #  Household Income
elif to_use == 'Box-Cox':
    house_pr_score_01 = df_transformed['hpb_01']  #  House Prices
    hh_inc_score_01   = df_transformed['hhb_01']  #  Household Income
    house_pr_score_11 = df_transformed['hpb_11']  #  House Prices
    hh_inc_score_11   = df_transformed['hhb_11'] 
elif to_use == 'Log':
    house_pr_score_01 = df_transformed['hpl_01']  #  House Prices
    hh_inc_score_01   = df_transformed['hhl_01']  #  Household Income
    house_pr_score_11 = df_transformed['hpl_11']  #  House Prices
    hh_inc_score_11   = df_transformed['hhl_11'] 

### Safety Checks

We can't have NaN/Non-Finite values for PCA processing, so although we could fill in missing values by taking a weighted mean of the surrounding LSOAs wherever values are missing, the more effective (and less problematic) way is simply to drop them. Note that this means the data sets are _not_ the same size.

In [ ]:
checks = {
    "Qualifications 2001":quals_score_01,
    "Qualifications 2011":quals_score_11,
    "Occupations 2001":occ_score_01,
    "Occupations 2011":occ_score_11,
    "House Prices 2001":house_pr_score_01,
    "House Prices 2011":house_pr_score_11,
    "Incomes 2001":hh_inc_score_01,
    "Incomes 2011":hh_inc_score_11
}

for k, v in checks.items():
    if (np.isnan(v.values).any()):
        print("Have null values in data set: " + k)

In [ ]:
#  Create dataset of indicator data - 2001
res_01 = pd.concat([house_pr_score_01,quals_score_01,occ_score_01,hh_inc_score_01], axis=1)
res_11 = pd.concat([house_pr_score_11,quals_score_11,occ_score_11,hh_inc_score_11], axis=1)

if to_use is 'Untransformed':
    res_01.columns = ['House Prices 2001','Percentage with Level 4+ Qualifications 2001',
                      'Percentage of Knowledge Workers 2001','Household Income 2001']
    res_11.columns = ['House Prices 2011','Percentage with Level 4+ Qualifications 2011',
                      'Percentage of Knowledge Workers 2011','Household Income 2011']
else:
    res_01.columns = ['House Prices 2001 (' + to_use + ' Transformed)','Percentage with Level 4+ Qualifications 2001',
                      'Percentage of Knowledge Workers 2001','Household Income 2001 (' + to_use + ' Transformed)']
    res_11.columns = ['House Prices 2011 (' + to_use + ' Transformed)','Percentage with Level 4+ Qualifications 2011',
                      'Percentage of Knowledge Workers 2011','Household Income 2011 (' + to_use + ' Transformed)']

# Create dataset of indicator data
X_01 = res_01.values
X_11 = res_11.values

#  Join 2001 and 2011 datasets and sanity-check
SES_inds = np.concatenate((X_01, X_11), axis=0)

print("Any infinite values? " + str(~np.isfinite(SES_inds).any()))
print("Any NaN values? " + str(np.isnan(SES_inds).any()))

In [ ]:
#  Median removal and Unit scaling
scaler = preprocessing.RobustScaler()
scaler.fit(SES_inds)
SES_inds = scaler.transform(SES_inds)

print("Data scaled and transformed.")

This next section just gives us a sense of loadings and how much each component counts if we were performing full PCA:

Explained Variance:
* _Untransformed_: `[0.78793941, 0.151054, 0.04878766, 0.01221892]`
* _Box Cox_: `[0.78728497, 0.15370062, 0.03948151, 0.01953289]`
* _Log_: `[0.79813204, 0.14576833, 0.03743246, 0.01866718]`

In [ ]:
pca_full = decomposition.PCA()                           # Use all Principal Components
pca_full.fit(SES_inds)                                   # Train model on data
SES_full_T = pd.DataFrame(pca_full.transform(SES_inds))  # Transform data using model

print("The amount of explained variance of the SES score using each component is...")
print(pca_full.explained_variance_ratio_)

# Adapted from https://stackoverflow.com/questions/22984335/recovering-features-names-of-explained-variance-ratio-in-pca-with-sklearn
i = np.identity(SES_inds.shape[1])  # identity matrix

coef = pca_full.transform(i)

loadings = pd.DataFrame(coef, index=res_01.columns)
loadings.to_csv(os.path.join(scores,to_use + '-Loadings-2011.csv.gz'), compression='gzip', index=True)

This is the transform we'll actually use. Notice that we limit `n_components` to 1 since we can only have a single score in our prediction code. So a PCA model is fitted and the transformation for the first component is used to assign each LSOA a score, before the 2001 and 2011 results are separated.

I make it that the explained variances for each approach are:
* _Untransformed_: 0.78794
* _Box Cox_: 0.78728
* _Natural log_: 0.79813

In [ ]:
#  Fitting PCA Model to derive SES score
pca = decomposition.PCA(n_components=1)             # Only need 1st Principal Component
pca.fit(SES_inds)                                   #  Train model on data
SES_inds_T = pd.DataFrame(pca.transform(SES_inds))  #  Transform data using model

print("The amount of explained variance of the SES score is: {0:6.5f}".format(pca.explained_variance_ratio_[0]))

In [ ]:
#  Split transformed data into 2001 and 2011 datasets
#  Note the way we do this to deal with missing data (if any)
scores_01 = SES_inds_T.loc[0:len(X_01)-1,0]
scores_11 = SES_inds_T.loc[len(X_01):,0]

# Create dfs from the two sets of scores
res_01 = res_01.assign(scores=pd.Series(scores_01).values)
res_11 = res_11.assign(scores=pd.Series(scores_11).values)

#res.columns = ['LSOANM','PRICE-01','QUALS-01','OCC-01','INCOME-01','PRICE-11',
#               'QUALS-11','OCC-11','INCOME-11','SES_01','SES_11']

# Join them together so we've got a single df for 2001 and 2011
res = res_01.merge(res_11, how='outer', suffixes=('_01','_11'), left_index=True, right_index=True)

# Rename columns for consistency with Jordan's code
res.rename(columns={'scores_01':'SES_01', 'scores_11':'SES_11'}, inplace=True)

# Sanity check
res.head(3)

The below code computes other metrics for the LSOAs including:  SES ascent, SES percentile ascent.

In [ ]:
#  Compute rank of LSOA in 2001 (so low rank = 'low status')
res['RANK_01'] = res.SES_01.rank(ascending=False)

#  Compute rank of LSOA in 2011 (so low rank = 'low status')
res['RANK_11'] = res.SES_11.rank(ascending=False)

#  Compute amount by which LSOA has ascended (so +ve = status improvement; -ve = status decline)
res.loc[:,'SES_ASC'] = res.loc[:,'SES_11'] - res.loc[:,'SES_01']

In [ ]:
import re 
#  Calculate LSOA percentile score in 01
res.loc[:,'SES_PR_01'] = res.RANK_01.rank(ascending=False, pct=True) * 100

#  Calculate LSOA percentile score in 11
res.loc[:,'SES_PR_11'] = res.RANK_11.rank(ascending=False, pct=True) * 100

#  Calculate percentile change (so +ve = 'moved up' in the world; -ve = 'moved down')
res.loc[:,'SES_PR_ASC'] = res.loc[:,'SES_PR_11'] - res.loc[:,'SES_PR_01']

inp = res.loc[:,[x for x in res.columns if 'SES' not in x and 'RANK' not in x]]

# Tidy up the naming
inp.rename(columns=lambda x: re.sub('_11',' 2011',re.sub('_01',' 2001',x)), inplace=True)
inp.rename(columns=lambda x: re.sub('kw_pct','Knowledge Worker Percentage',x), inplace=True)
inp.rename(columns=lambda x: re.sub('he_pct','Highly-Educated Percentage',x), inplace=True)
inp.rename(columns=lambda x: re.sub('hp','Property Prices (Transformed)',x), inplace=True)
inp.rename(columns=lambda x: re.sub('hh','Household Income (Transformed)',x), inplace=True)

# Save to file (note that we are also saving some info about the input variables as we use these as well)
res[
    ['RANK_01','RANK_11','SES_01','SES_11','SES_ASC','SES_PR_01','SES_PR_11','SES_PR_ASC']
].to_csv(os.path.join(analytical,to_use + '-Scores.csv.gz'), compression='gzip', index=True) 
inp[
    [x for x in inp.columns if '2001' in x]
].to_csv(os.path.join(scores,to_use + '-Inputs-2001.csv.gz'), compression='gzip', index=True)
inp[
    [x for x in inp.columns if '2011' in x]
].to_csv(os.path.join(scores,to_use + '-Inputs-2011.csv.gz'), compression='gzip', index=True)

## Diagnostics

If you are comfortable with the output of the code above you do not need to run the blocks below as these are simply sanity checks to help you (me) envision the output effectively.

In [ ]:
#  Sanity check
res[['SES_01','SES_11','RANK_01','RANK_11','SES_PR_01','SES_PR_11','SES_PR_ASC']].sample(5, random_state=r_state)

In [ ]:
# The lowest-ranked (highest status) LSOAs
res.loc[res['RANK_01'] < 5,:].sort_values('RANK_01')

In [ ]:
# The highest-ranked (lowest status) LSOAs
res.loc[res['RANK_01'] > (res.RANK_01.max()-5),:].sort_values('RANK_01')

In [ ]:
# Biggest falls in percentile status
res.sort_values('SES_PR_ASC').head(5)

In [ ]:
# Biggest gains in percentile status
res.sort_values('SES_PR_ASC', ascending=False).head(5)

In [ ]:
g = sns.jointplot(x='SES_01', y='SES_11', data=res, kind='scatter', s=3, color='k', height=7, ratio=5, space=0, linewidth=1)

In [ ]:
g = sns.jointplot(x='SES_PR_01', y='SES_PR_11', data=res, kind='scatter', s=3, color='k')

In [ ]:
g = sns.jointplot(x='RANK_01', y='RANK_11', data=res, kind='scatter', s=3, color='k')

### Automated Mapping

With a little tweaking (as a result of changes I made and didn't propagate) you could automatically map the results in Python directly; however, I found that the overall results were rather better in QGIS and so haven't updated [notebook 10](10-Mapping Scores.ipynb).